In [4]:
import pandas as pd
import boto3
import subprocess
import json
import os

In [5]:
def csv_to_geojson(csv_file, attr_lat, attr_lon, attr_elev,output_geojson_file):
    """
    Converts a CSV file with latitude, longitude, and altitude to a GeoJSON file with EPSG:7844 CRS.
    All other columns in the CSV are included as properties in the GeoJSON features.
    
    Args:
        csv_file (str): Path to the input CSV file.
        output_geojson_file (str): Path to the output GeoJSON file.
    """
    try:
        # Read the CSV file
        df = pd.read_csv(csv_file)
        
        # Validate required columns
        required_columns = [attr_lat, attr_lon, attr_elev]
        if not all(column in df.columns for column in required_columns):
            raise ValueError(f"CSV file must contain the following columns: {required_columns}")
        
        # Create a list to hold the GeoJSON features
        features = []
        
        # Iterate over each row in the DataFrame
        for index, row in df.iterrows():
            # Extract latitude, longitude, and altitude
            lat = row[attr_lat]
            lon = row[attr_lon]
            altitude = row[attr_elev]
            
            # Create a GeoJSON Point geometry
            point = {
                "type": "Point",
                "coordinates": [lon, lat, altitude]
            }
            
            # Create a properties dictionary with all other columns
            properties = row.drop([attr_lat, attr_lon, attr_elev]).to_dict()
            
            # Create a GeoJSON feature with the point and properties
            feature = {
                "type": "Feature",
                "geometry": point,
                "properties": properties
            }
            
            # Add the feature to the list
            features.append(feature)
        
        # Create a GeoJSON FeatureCollection
        feature_collection = {
            "type": "FeatureCollection",
            "features": features,
            "crs": {
                "type": "name",
                "properties": {
                    "name": "urn:ogc:def:crs:EPSG::7844" # update to match the data CRS
                }
            }
        }
        
        # Write the FeatureCollection to a GeoJSON file
        with open(output_geojson_file, 'w') as f:
            json.dump(feature_collection, f, indent=4)
        
        print(f"GeoJSON file saved to {output_geojson_file}")
    
    except Exception as e:
        print(f"An error occurred: {e}")

In [6]:
csv_file = 'output/panorama/FramePosOptimised-wagga-wagga.csv'  # output CSV file path
output_geojson_file = 'output/panorama/FramePosOptimised-wagga-wagga_EPSG7844.geojson'  # output GeoJSON file path

s3_bucket = 'frontiersi-p127-floor-height-woolpert'
object_key='01_WaggaWagga/03_Ancillary/02_TrajectoryFiles/rev1/FramePosOptimised-wagga-wagga.csv'


In [7]:
# set up connection to s2 bucket
mfa_token = '821214'
# cmd = (f'C:\\Program Files\\Amazon\\AWSCLIV2\\aws sts get-session-token --serial-number arn:aws:iam::693903849513:mfa/Lavender_AWS_MFA  --token-code {mfa_token}').split()
cmd = (f'aws sts get-session-token --serial-number arn:aws:iam::693903849513:mfa/Lavender_AWS_MFA  --token-code {mfa_token}').split()
result = subprocess.run(cmd, capture_output=True)
# Check if the command was successful
if result.returncode == 0:
    # Parse JSON output into a dictionary
    data = json.loads(result.stdout)
    #print(data)  # Print or process the dictionary
else:
    print("Error:", result.stderr)

session = boto3.session.Session(
    aws_access_key_id=data['Credentials']['AccessKeyId'],
    aws_secret_access_key=data['Credentials']['SecretAccessKey'],
    aws_session_token=data['Credentials']['SessionToken']
)
# Initialize an S3 client
s3_client = session.client('s3')


In [8]:
# download trajectory file
s3_client.download_file(s3_bucket, object_key, csv_file)


In [9]:
# convert to csv
csv_to_geojson(csv_file, 'Latitude_deg', 'Longitude_deg', 'Altitude_m', output_geojson_file)

GeoJSON file saved to output/panorama/FramePosOptimised-wagga-wagga_EPSG7844.geojson
